# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [92]:
#Your code here
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.stats.weightstats as sm
from scipy import stats
%matplotlib inline
sns.set_style('dark')

data = pd.read_csv('homepage_actions.csv')
data.head()

,timestamp,id,group,action
0,2016-09-24 17:42:27.839496,804196,experiment,view
1,2016-09-24 19:19:03.542569,434745,experiment,view
2,2016-09-24 19:36:00.944135,507599,experiment,view
3,2016-09-24 19:59:02.646620,671993,control,view
4,2016-09-24 20:26:14.466886,536734,experiment,view


In [93]:
# views vs clicks
data.action.value_counts()

view     6328
click    1860
Name: action, dtype: int64

In [94]:
# clicks but no views
clicks = data[data['action'] == 'click']
views = data[data['action'] == 'view']

anomaly = []
for value in clicks.id.values:
    if value not in views.id.values:
        anomaly.append(value)
print('number of clicks but no views: ', len(anomaly))

number of clicks but no views:  0


In [95]:
# overlap between control and experiment groups
cont = data[data['group'] == 'control']     
exp = data[data['group'] == 'experiment'] 

a = set(cont['id'].values).intersection(set(exp['id'].values)) 
print('no overlap between control and experiment group', a)

no overlap between control and experiment group set()


## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [108]:
#Your code here
#transform the action column

data['count'] = 1

control = data[data.group=='control'].pivot(index='id', columns='action', values='count')
control = control.fillna(value=0)

experiment = data[data.group=='experiment'].pivot(index='id', columns='action', values='count')
experiment = experiment.fillna(value=0)


print("Sample sizes:\tControl: {}\tExperiment: {}".format(len(control), len(experiment)))
print("Total Clicks:\tControl: {}\tExperiment: {}".format(control.click.sum(), experiment.click.sum()))
print("Average click rate:\tControl: {}\tExperiment: {}".format(control.click.mean(), experiment.click.mean()))

Sample sizes:	Control: 3332	Experiment: 2996
Total Clicks:	Control: 932.0	Experiment: 928.0
Average click rate:	Control: 0.2797118847539016	Experiment: 0.3097463284379172


* *Null hypothesis*: there is no difference in the number of clicks between the control & experimental groups
* *Alternative hypothesis*: there is a difference in the number of clicks between the control & experimental groups

In [114]:
# assuming unequal variance, we'll use welch t-test

result = sm.ttest_ind(control.click, experiment.click, usevar='unequal') # set usevar to unequal for welch
t_statistic, p_value, df = result

if p_value < 0.05:
    print('Null hypotheis rejected')
    print('the number of clicks in the experimental group is significantly lower')
else:
    print('No sufficient evidence to reject the null hypothesis')
    print('No difference in the number of clicks between control & experimental groups')


Null hypotheis rejected
the number of clicks in the experimental group is significantly lower


## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [117]:
#Your code here
n = experiment.shape[0]
p = control.click.mean()

expct_clicks_exper = n * p

print('experiment expected # of clicks: ', round(expct_clicks_exper, 2))


experiment expected # of clicks:  838.02


### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [119]:
#Your code here
std = round((expct_clicks_exper * (1 - p))**0.5, 2)
std

24.57

### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [121]:
#Your code here

z = (experiment.click.sum() - expct_clicks_exper) / std
print(z)

pval = stats.norm.sf(z)
print(pval)

3.6623196287061814
0.00012497086532602507


### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: Yes, while the p-value is slightly lower, both would lead to confident rejection of the null hypothesis. The experimental page appears to be a more effective design.

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.